In [ ]:
from game.individuals.dot import Dot

from random import choice, uniform
from copy import copy

import numpy as np

class Breeder:
    def __init__(self, parent):
        self.parent = parent

    def breed(self, population):
        
        return self.breed_function(population)

    def initialize_population(self, num_individuals, color):
        
        return self.initialize_population_function(num_individuals, color)
    
    def initialize_population_function(self, num_individuals, color):

        population = []
        for _ in range(num_individuals):
            population.append(Dot(self.parent, color=(0,139,139))) #Here I could just create random dna for num_individuals
            
        return population
    
    def breed_function(self, population):
       
        population_cpy = copy(population)
        dead = []
        alive = []
        for individual in population_cpy:
            if individual.dead:
                dead.append(individual)
            else:
                alive.append(individual)

        for _ in range(len(dead)):
            # get the position where the child should be inserted on the field
            where = choice(alive)._position
            color = alive[0].color

            selected = self.select(population_cpy) #con mi select siempre cojo los mismos padres
            parent1 = selected[0]
            parent2 = selected[1]
            child1, child2 = self.crossover_example(copy(parent1), copy(parent2)) 
            child1 = self.tweak_example(child1)
            child2 = self.tweak_example(child2)
            score_child1 = self.assess_individual_fitness(child1) #Frames survived and food eaten = parent1 and 2 so the crossover and tweak are not doing anything
            score_child2 = self.assess_individual_fitness(child2)
            if score_child1 > score_child2:
                new_individual = Dot(self.parent, color=color, position=where, dna=child1.get_dna())
                print(score_child1)
            else:
                new_individual = Dot(self.parent, color=color, position=where, dna=child2.get_dna())
            population_cpy.append(new_individual)
        for dead_individual in dead:
            population_cpy.remove(dead_individual)
        return population_cpy
    
    def select(self, population):
        
        fitness_array = np.empty([len(population)])
        for i in range(len(population)):
            score = self.assess_individual_fitness(population[i])
            fitness_array[i] = score
        
        # span value range
        for i in range(1, len(fitness_array)):
            fitness_array[i] = fitness_array[i] + fitness_array[i - 1]
        
        parents = self.selectParentSUS(population, fitness_array, 2)
        return parents
    
    def my_select(population, best, best_score):
        total_fitness_array = []
        parents = []
        population_best = []
        population_best_score = 0
        for i in range(len(population)):
            next = population[i]
            total_fitness = assess_fitness(next)
            total_fitness_array.append(total_fitness)
            if(i==0):
                population_best = next
                population_best_score = total_fitness
            elif(total_fitness > population_best_score):
                population_best = next
                population_best_score = total_fitness

        if population_best_score > best_score:
            best = population_best
            best_score = population_best_score

        fitness_dictionary = dict(zip(total_fitness_array, population))
        sorted_tuples = sorted(fitness_dictionary.items(), key= itemgetter(0), reverse = True)
        parents = [item[1] for item in sorted_tuples][:breed_number] #select the top bree_number parents by the total_fitness_array
        return parents, best, best_score
    
    def assess_individual_fitness(self, individual):
        
        statistic = individual.statistic
        dna = individual.get_dna()
        score = statistic.time_survived * statistic.food_eaten
        return score
    
    def selectParentSUS(self, population, fitness_array, count): #Fitness proportional selection
        
        individual_indices = []
        # build the offset = random number between 0 and f_l / n
        offset = uniform(0, fitness_array[-1] / count) #[-1] is the last item
        # repeat for all selections (n)
        for _ in range(count):
            index = 0
            # increment the index until we reached the offset
            while fitness_array[index] < offset:
                index += 1
            # increment the offset to the next target
            offset = offset + fitness_array[-1] / count
            individual_indices.append(population[index]) #SELECT the index of the desired individual
        # return all selected individual indices
        return np.array(individual_indices)
    
    def population_create (population_size):
        population = []
        l = 0
        while l < population_size:
            population.append(np.random.randint(2,size = np.size(names_vector)))
            l+=1
        return population

    def crossover_example(self, solution_a, solution_b):
        
        dna_a = solution_a.get_dna()
        dna_b = solution_b.get_dna()
        for i in range(len(dna_a)):
            if uniform(0, 1) < 0.5:
                tmp = dna_a[i]
                dna_a[i] = dna_b[i]
                dna_b[i] = tmp
        solution_a.dna_to_traits(dna_a)
        solution_b.dna_to_traits(dna_b)
        return solution_a, solution_b
    
    def crossover(self, solution_a, solution_b):
        dna_a = solution_a.get_dna()
        dna_b = solution_b.get_dna()
        k = random.randint(1, len(features)-1) #one point crossover
        #swap with separator at k
        temp = copy(dna_b[0:k])
        dna_b[0:k] = copy(dna_a[0:k])
        dna_a[0:k] = copy(temp)
        solution_a.dna_to_traits(dna_a)
        solution_b.dna_to_traits(dna_b)
        return solution_a, solution_b

    def mutation(population):
        new_population = []

        for individual in population:
            for(i) in range(len(individual)):
                generator = random.random()
                if generator < flip_probability:
                    individual[i] = 1 - (individual[i])
            new_population.append(individual)
        return new_population
    
    def tweak_example(self, individual):

        dna = individual.get_dna()
        increase = uniform(0, 0.3) #puedo coger mas valores y puedo repetir el proceso dos o tres veces.
    
        perc = dna[0]
        des = dna[1]
        abil = dna[2]
        
        best_perc = np.argmax(perc)
        best_des = np.argmax(des)
        best_abil = np.argmax(abil)
        
        worst_perc = np.argmin(perc)
        worst_des = np.argmin(des)
        worst_abil = np.argmin(abil)
        
        perc = self.my_mutate_dna(dna=perc, increase_value=increase, increase=best_perc, decrease=worst_perc) #I can chose another one to be increased RANDOM
        des = self.my_mutate_dna(dna=des, increase_value=increase, increase=best_des, decrease=worst_des)
        abil = self.my_mutate_dna(dna=abil, increase_value=increase, increase=best_abil, decrease=worst_abil)

        dna = [perc, des, abil]
        individual.dna_to_traits(dna)
        return individual

    def mutate_dna(self, dna, increase_value, increase, decrease): #decreases a increase_value from one random feature and increases the selected in increse=0
        # select some other dna to be decreased
        choices = [i for i in range(len(dna))]
        choices.remove(increase)
        decreased = False
        while not decreased:
            decrease = choice(choices)
            if dna[decrease] - increase_value >= 0.0:
                dna[decrease] -= increase_value
                decreased = True
            else:
                choices.remove(decrease)
            if len(choices) == 0:
                break
        # if we were able to reduce the value for the other dna -> increase the desired dna
        if decreased:
            # increase the value
            dna[increase] += increase_value if dna[increase] <= 1.0 else 1.0 #Y la suma total seria menor q 1
        # otherwise we cannot do anything
        return dna
    
     def my_mutate_dna(self, dna, increase_value, increase, decrease): #decreases a increase_value from one random feature and increases the selected in increse=0
        # select some other dna to be decreased
        while not increased:
            if dna[increase] + increase_value > 1:
                increase_value = 1-dna[increase]
                dna[increase] = 1
            else:
                dna[increase] += increase_value
            
            dna[decrease] -= increase_value
            if dna[decrease] < 0:
                left_over=0-dna[decrease]
                dna[decrease] = 0
            choices = [i for i in range(len(dna))]
            choices.remove(increase)
            choices.remove(decrease)
            decrease2 = random.choice(choices)
            decrease2-=left_over
        return dna
    

In [ ]:
#Hints
#DNA This is how the DNA of an individual looks like:

# dna = [perception, desires, abilities]
# perception = [food, poison, health_potion, opponent, corpse, predator]
# desires = [seek_food, dodge_poison, seek_potions, seek_opponents, seek_corpses, dodge_predators]
# abilities = [armor, speed, strength, poison_resistance, toxicity]

#dna = [[0.3, 0.7, 0, 0, 0, 0], [0.3, 0, 0.6, 0.1, 0, 0], [0.3, 0.2, 0, 0, 0.5]]
#Dead or Alive Individuals have a bool value named dead. If it is True, the individual is dead.

#Statistics You have access to individuals statistics. It consist of:

#time survived (in amount of frames)
#food eaten
#poison eaten
#consumed potions
#consumed corpses
#enemies attacked
#attacked by opponents
#attacked by predators
#food seen
#poison seen
#potions seen
#opponents seen
#predators seen
#corpses seen

In [2]:
#import numpy as np

#perc = [0,3,4,7,1,20,4,6,8,45,2,23,15,16,49,3]
#best_perc = np.argmax(perc)
#print(best_perc)

14
